In [1]:
'''
OpenFDA Label Query for basic DP infomation. 
Optimized for Speed
Nguyen D. Vu
'''

'\nOpenFDA Label Query for basic DP infomation. \nOptimized for Speed\nNguyen D. Vu\n'

In [2]:
# Import Necessary Modules (some may not be immediately necessary )
import os
import numpy as np
import pandas as pd
import requests
import json
import itertools
import re

In [ ]:
def API_list(x):
    pass

In [3]:
# list of drugs to test
fdg_target = "midazolam,Propofol,cisatracurium,ciprofloxacin,Albuterol,Desloratadine,Donepezil,Ezetimibe,Glimepiride,Glipizide,Meloxicam,Metoclopramide,Ondansetron,Rivastigmine,Tolterodine".lower()

target_drugs_ls = fdg_target.split(",")

In [4]:
def json_get(key_str, json_obj):
    try:
        return json_obj[key_str][0]
    except (KeyError, AttributeError):
        return 'Not_Found'
    

In [5]:
#define search Parameter on Label DB on Open FDA

#Enpoint to search for API from label
url_root = "https://api.fda.gov/drug/label.json?"
APIkey = "api_key=ieSIcZGsM6K0XVG6kL7UiQb4Yw3cDN1z2bwO4XPl"

# Define search field
field = "openfda.substance_name:"



In [6]:
stored_results = {
    "API":[],
    "Description":[],
    "SPL":[],
    "openFDA":[]
}


for API in target_drugs_ls:
    # define method to limit and skip through pages of query up to 26,000 results.
    lim = 1000
    skip = 0
 

    while True:
        lim_str = "&limit="
        skp_str = "&skip="
        query = url_root+APIkey+"&search="+field+API+lim_str+str(lim)+skp_str+str(skip)
        try:
            req = requests.get(query)
            req.raise_for_status()

            #load Results to Json
            js_read = req.json()
            drug = js_read['results']
            desc = [json_get(key_str='description', json_obj=i) for i in drug]
            spl_elements = [json_get(key_str='spl_product_data_elements', json_obj=i) for i in drug]
            open_fda = [i['openfda'] for i in drug]
            
            stored_results['API'].extend([API for i in drug])
            stored_results['Description'].extend(desc)
            stored_results['SPL'].extend(spl_elements)
            stored_results['openFDA'].extend(open_fda)
            
            
            skip += lim
        except requests.exceptions.HTTPError as e:
            print ("No more matches found for {}".format(API))
            break




No more matches found for midazolam
No more matches found for propofol
No more matches found for cisatracurium
No more matches found for ciprofloxacin
No more matches found for albuterol
No more matches found for desloratadine
No more matches found for donepezil
No more matches found for ezetimibe
No more matches found for glimepiride
No more matches found for glipizide
No more matches found for meloxicam
No more matches found for metoclopramide
No more matches found for ondansetron
No more matches found for rivastigmine
No more matches found for tolterodine


In [7]:
#get the list of interested columns in open FDA section
openfda_keys = []
for i in (stored_results['openFDA']):
    openfda_keys.extend(i.keys())

openfda_keys = list(set(openfda_keys))

# print to decide the column of interest
print(openfda_keys)


['spl_set_id', 'is_original_packager', 'spl_id', 'package_ndc', 'product_ndc', 'brand_name', 'product_type', 'pharm_class_moa', 'pharm_class_cs', 'pharm_class_epc', 'substance_name', 'pharm_class_pe', 'upc', 'application_number', 'nui', 'unii', 'route', 'manufacturer_name', 'generic_name', 'original_packager_product_ndc', 'rxcui']


In [8]:
#define columns to extract from the open_fda portions:
openfda_list = ['package_ndc', 'unii', 'manufacturer_name', 'pharm_class_pe', 'product_type', 'brand_name', 'substance_name', 'pharm_class_cs', 'pharm_class_epc', 'route']
openfda_dict = {i:[] for i in openfda_list}

for i in stored_results['openFDA']:
    for key in openfda_dict:
        try:
            openfda_dict[key].append(i[key])
        except (KeyError, AttributeError):
            openfda_dict[key].append(["No_Info"])
    

In [9]:
openfda_dict.keys()

dict_keys(['package_ndc', 'unii', 'manufacturer_name', 'pharm_class_pe', 'product_type', 'brand_name', 'substance_name', 'pharm_class_cs', 'pharm_class_epc', 'route'])

In [10]:
#combine dictionary

print(type(stored_results))
print(type(openfda_dict))

data = {**stored_results, **openfda_dict}

<class 'dict'>
<class 'dict'>


In [11]:
data.pop('openFDA')
df_ = pd.DataFrame.from_dict(data)

In [12]:
print(df_.shape)
df_.head()

(1192, 13)


,API,Description,SPL,package_ndc,unii,manufacturer_name,pharm_class_pe,product_type,brand_name,substance_name,pharm_class_cs,pharm_class_epc,route
0,midazolam,DESCRIPTION Midazolam hydrochloride is a water...,MIDAZOLAM midazoalm MIDAZOLAM HYDROCHLORIDE MI...,[52584-749-01],[W7TTW573JJ],"[General Injectables and Vaccines, Inc.]",[No_Info],[HUMAN PRESCRIPTION DRUG],[MIDAZOLAM],[MIDAZOLAM HYDROCHLORIDE],[No_Info],[No_Info],"[INTRAMUSCULAR, INTRAVENOUS]"
1,midazolam,DESCRIPTION: Midazolam hydrochloride is a wate...,Midazolam Midazolam Hydrochloride MIDAZOLAM HY...,[63323-412-00],[W7TTW573JJ],"[Fresenius Kabi USA, LLC]",[No_Info],[HUMAN PRESCRIPTION DRUG],[Midazolam],[MIDAZOLAM HYDROCHLORIDE],[No_Info],[No_Info],"[INTRAMUSCULAR, INTRAVENOUS]"
2,midazolam,DESCRIPTION Midazolam is a benzodiazepine avai...,Midazolam Hydrochloride Midazolam Hydrochlorid...,[0574-0150-04],[W7TTW573JJ],"[Paddock Laboratories, LLC.]",[No_Info],[HUMAN PRESCRIPTION DRUG],[Midazolam Hydrochloride],[MIDAZOLAM HYDROCHLORIDE],[No_Info],[No_Info],[ORAL]
3,midazolam,DESCRIPTION Midazolam hydrochloride injection ...,Midazolam Hydrochloride MIDAZOLAM HYDROCHLORID...,"[0409-2305-16, 0409-2305-17, 0409-2305-02, 040...",[W7TTW573JJ],"[Hospira, Inc.]",[No_Info],[HUMAN PRESCRIPTION DRUG],[Midazolam Hydrochloride],[MIDAZOLAM HYDROCHLORIDE],[No_Info],[No_Info],"[INTRAMUSCULAR, INTRAVENOUS]"
4,midazolam,"DESCRIPTION: Midazolam Injection, USP is a wat...",Midazolam Midazolam Hydrochloride SODIUM CHLOR...,"[76045-211-00, 76045-211-20, 76045-001-00, 760...",[W7TTW573JJ],"[Fresenius Kabi USA, LLC]",[No_Info],[HUMAN PRESCRIPTION DRUG],[Midazolam],[MIDAZOLAM HYDROCHLORIDE],[No_Info],[No_Info],"[INTRAMUSCULAR, INTRAVENOUS]"


In [15]:
data.keys()

dict_keys(['API', 'Description', 'SPL', 'package_ndc', 'unii', 'manufacturer_name', 'pharm_class_pe', 'product_type', 'brand_name', 'substance_name', 'pharm_class_cs', 'pharm_class_epc', 'route'])

In [ ]:
#Data Clean up methods:

# Define a method to clean up the ingredients list:
def getIngredients(x, active_list):
    if type(x) is list:
        words = "".join(x).upper().split(" ")
#         words = [v for i,v in enumerate(words) if i==0 or v!=words[i-1]]

        api_words = " ".join(active_list).upper()
        words = [v for v in words if v not in api_words]
        words = list(set(words))
        #ingredients_list = " ".join(words)
        
        return words
    
    else:
        print("input must be a list")


# Define a method to flatten out the label NDc list
def flatNDC(x):
    if type(x) is list:
        merged = list(itertools.chain(*x))
    else:
        merge = x
    return merge


# Define a method to extract concentrations based on what is in spl_product_data_elements

def ingredientsConc(ingr_list, text):
    if type(ingr_list) is list:
        conc = []
        words = "".join(ingr_list).upper().split(" ")
        words = [v for i,v in enumerate(words) if i==0 or v!=words[i-1]]
        text_in = [text[0] if text==text else ""]
        text_ = "".join(text_in)
        for w in words:
            pattern = r"(\d+[(.?\d*\W)|(?:mg|g|ml)]+\s+{})".format(w)
            p = re.compile(pattern, re.IGNORECASE+re.M)
            conc.append(p.findall(text_))
        
        merged_conc = list(itertools.chain(*conc))
        merged_conc = [v for i,v in enumerate(merged_conc) if i==0 or v!=merged_conc[i-1]]
        return merged_conc
    
    else:
        print("Ingredient List must be a list")
        
# define a method for storage temp extract

def getStorage(storage_and_handling):
    if type(storage_and_handling) is list:

        words = "".join(storage_and_handling).lower()
        
        pattern_storeage = r"(?<=store\sat )(.*?)(?=\()"
        p = re.compile(pattern_storeage, re.IGNORECASE+re.M)
        store_temp  = p.findall(words)
        
        return store_temp
    
    else:
        pass

# define a method for extracting pH
def getpH(description):
    if type(description) is list:

        words = "".join(description)
        
        pattern_pH = r"(?<=pH\s).*?(?=\d)(\d*\W\d*\s+\w+\s+\d+\W\d+)"
        simp_pattern_pH = r"(?<=pH\s).*?(?=\d)(\d)"
        store_pH  = re.search(pattern_pH, words, re.MULTILINE)
        store_pH_simp  = re.search(simp_pattern_pH, words, re.MULTILINE)
        
        if store_pH is not None:
            return store_pH.group(1)
        if store_pH_simp is not None:
            return store_pH_simp.group(1)
    
    else:
        pass

# define a method for extracting Dose
def getDose(data_list):
    dose_info = []
    for l in data_list:
        if type(l) is list:
            words = "".join(l)
            pattern_dose = r"(multi\w*|single)(?=\-+dose)"
            store_dose  = re.search(pattern_dose, words, re.MULTILINE+re.IGNORECASE)
            if store_dose is not None:
                dose_info.append(store_dose.group(1))
        else:
            pass
    
    dose_out = ",".join(dose_info)
    return dose_out
